<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/spell/D_spell_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		 df_C_inc.csv	   df_K_inc.csv     spell_check
'Colab Notebooks'	'# df_G_inc.csv'   df_P_inc.csv
 C_ori_model.pt		 df_G_inc.csv	  'My Drive'
'답안 작성용 파일.csv'	 df_J_inc.csv	   Q_inc_model.pt


In [ ]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/1. 실습용자료.txt  


In [ ]:
import pandas as pd
df=pd.read_csv('/content/dataset/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")

## 데이터 전처리

In [ ]:
df=df[(df['digit_1'] =='D')] #조건에 맞는 데이터 출력

In [ ]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0001519,D,35,351,태양광,전기를 모아,태양력발전
1,id_0002425,D,35,351,태양광,전기생산,태양광발전소
2,id_0002666,D,35,351,태양광,전기생산,태양력발전업
3,id_0002762,D,35,351,태양광,전기생산,판매
4,id_0003695,D,35,351,태양광,전기생산,한전에판매
...,...,...,...,...,...,...,...
751,id_0992466,D,35,351,태양광이용,태양광발전,전기생산
752,id_0993939,D,35,351,태양광이용,태양광발전,태양광전기생산
753,id_0994334,D,35,351,태양광이용,탱양광발전,전기생산
754,id_0995674,D,35,352,사업장에서,도시가스를 배관시설에 의해,"가정,산업에 공급"


In [ ]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0001519,D,35,351,태양광 전기를 모아 태양력발전
1,id_0002425,D,35,351,태양광 전기생산 태양광발전소
2,id_0002666,D,35,351,태양광 전기생산 태양력발전업
3,id_0002762,D,35,351,태양광 전기생산 판매
4,id_0003695,D,35,351,태양광 전기생산 한전에판매
...,...,...,...,...,...
751,id_0992466,D,35,351,태양광이용 태양광발전 전기생산
752,id_0993939,D,35,351,태양광이용 태양광발전 태양광전기생산
753,id_0994334,D,35,351,태양광이용 탱양광발전 전기생산
754,id_0995674,D,35,352,"사업장에서 도시가스를 배관시설에 의해 가정,산업에 공급"


In [ ]:
df['digit_3'].value_counts()

351    660
352     67
353     29
Name: digit_3, dtype: int64

## 맞춤법 교정

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell

  Cloning https://github.com/ssut/py-hanspell to /tmp/pip-req-build-99ffj10w
  Running command git clone -q https://github.com/ssut/py-hanspell /tmp/pip-req-build-99ffj10w


In [ ]:
from hanspell import spell_checker

temp_result=[]
for sentence in df['text']:
  try:
    result=spell_checker.check(sentence)
    temp_result.append(result.checked)
  except:
    temp_result.append(sentence)

df['text']=pd.DataFrame(temp_result)  

In [ ]:
df.to_csv('df_D_spellcheck.csv', index=False, encoding='utf-8-sig')